# Analyzing Stackoverflow Data

We will explore and analyze dataset containing questions from writers.stackoverflow.com. 

Downloaded `writers.stackexchange.com.7z` specifically from https://archive.org/details/stackexchange

The schema documentation for the data can be found here for reference - https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede

In [1]:
import sys
sys.path.append('../')
from ml_editor.data_ingestion import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
pd.set_option('display.max_rows', 70)
pd.set_option('display.max_columns', 70)
pd.set_option('display.max_colwidth', 100)

## Download Data

In [2]:
site = "writers"
writers = get_data_from_dump(site)

## EDA

In [3]:
writers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41717 entries, 0 to 41716
Data columns (total 24 columns):
Unnamed: 0               41717 non-null int64
Id                       41717 non-null int64
PostTypeId               41717 non-null int64
AcceptedAnswerId         4971 non-null float64
CreationDate             41717 non-null object
Score                    41717 non-null int64
ViewCount                9674 non-null float64
Body                     41631 non-null object
OwnerUserId              38833 non-null float64
LastEditorUserId         13033 non-null float64
LastEditorDisplayName    985 non-null object
LastEditDate             13941 non-null object
LastActivityDate         41717 non-null object
Title                    9674 non-null object
Tags                     9674 non-null object
AnswerCount              9674 non-null float64
CommentCount             41717 non-null int64
FavoriteCount            3981 non-null float64
ClosedDate               1232 non-null object
Conten

More than 41K questions, but only ~9700 have answers.

In [4]:
writers.sample(5)

,Unnamed: 0,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,ContentLicense,body_text,ParentId,CommunityOwnedDate,OwnerDisplayName
11431,11431,16194,1,NaN,2015-02-11T21:36:56.037,2,119.0,"<p>I am writing a story that has ten main characters, each chapter is a progression of the story...",590.0,NaN,NaN,NaN,2015-02-12T03:31:15.817,Writing in a character's voice. How deep should I go with immersion?,<language><narrative><perspective>,1.0,1,NaN,NaN,CC BY-SA 3.0,"I am writing a story that has ten main characters, each chapter is a progression of the story bu...",NaN,NaN,NaN
13087,13087,18175,2,NaN,2015-07-13T12:03:34.027,0,NaN,"<p>I like technical advice, which is odd considering I enjoy poetry and poetry is something whic...",14494.0,6268.0,NaN,2015-07-14T02:08:11.020,2015-07-14T02:08:11.020,NaN,NaN,NaN,0,NaN,NaN,CC BY-SA 3.0,"I like technical advice, which is odd considering I enjoy poetry and poetry is something which o...",18172.0,NaN,NaN
7503,7503,8482,4,NaN,2013-07-24T02:22:24.207,0,NaN,Application programming interfaces (APIs) are software interfaces that expose defined functional...,1993.0,1993.0,NaN,2013-07-24T02:22:24.207,2013-07-24T02:22:24.207,NaN,NaN,NaN,0,NaN,NaN,CC BY-SA 3.0,Application programming interfaces (APIs) are software interfaces that expose defined functional...,NaN,NaN,NaN
8550,8550,9715,2,NaN,2013-12-13T20:09:06.050,1,NaN,<p>Several possibilities:</p>\n\n<p>1) Your (second set of?) questions turned out to be more tim...,5956.0,NaN,NaN,NaN,2013-12-13T20:09:06.050,NaN,NaN,NaN,0,NaN,NaN,CC BY-SA 3.0,Several possibilities:\n1) Your (second set of?) questions turned out to be more time-consuming ...,9706.0,NaN,NaN
22420,22420,29858,1,29862.0,2017-08-21T18:03:45.543,48,13139.0,"<p>I'm new here, so, please, bear with me.</p>\n\n<p>I had a look at previous answers to similar...",25977.0,-1.0,NaN,2017-08-22T06:45:58.447,2017-08-24T11:11:47.943,"I wrote a novel, now what?",<publishing><novel><editing><fantasy><science-fiction>,10.0,14,16.0,NaN,CC BY-SA 3.0,"I'm new here, so, please, bear with me.\nI had a look at previous answers to similar questions, ...",NaN,NaN,NaN


In [5]:
writers[writers["ViewCount"].notnull()]["ViewCount"].sample(5)

5679      137.0
24586     493.0
466      1783.0
21542      67.0
33934     265.0
Name: ViewCount, dtype: float64

In [6]:
notnull_titles = writers[writers["Title"].notnull()]["Title"]
notnull_titles.sample(5)

24135                                                                               What is a DF manuscript?
3459                                                             All persons fictitious - where to place it?
9344     Does self publishing via Amazon or similar services make your book ineligible for later acceptan...
11834                                                       What is the main inner conflict in Harry Potter?
5643                                                Which language to use when writing a multinational story
Name: Title, dtype: object

In [8]:
short_titles = notnull_titles[notnull_titles.str.len() < 20]
print(short_titles.shape)
short_titles.sample(5)

(175,)


21783       Daily Word Count
15218    Title already taken
12477       Renting my book?
26045      Writing diversity
15472     A ballad or prose?
Name: Title, dtype: object

In [9]:
no_body_text = writers[writers["body_text"].str.len() < 15]["body_text"]
print(no_body_text.shape)
no_body_text.sort_values(ascending=False).head(4)

(1,)


3375    \n
Name: body_text, dtype: object

In [10]:
body_text = writers[writers["body_text"].str.len() > 15]["body_text"]
print(body_text.shape)
body_text.sort_values(ascending=False).head(4)

(41630,)


17600    “Where do you get your inspiration?” \nThis is an often hated, and feared, Q author’s get. Their...
8343     “Out, Out—” has its morbid description of a young boy bleeding out and its underlying theme of d...
24097    “One should try to invite people from this world to eternity, from sin to obedience, from greedi...
10069    “If once a man indulges himself in murder, very soon he comes to think little of robbing; and fr...
Name: body_text, dtype: object

In [11]:
short_body_text = writers[(writers["body_text"].str.len() > 15) & (writers["body_text"].str.len() < 40)]["body_text"]
print(short_body_text.shape)
short_body_text.sort_values(ascending=False).head(4)

(53,)


2523       what about\n\nInformation IS Power\n\n
30953    it's called "as if told" first person.\n
2452                 help! I need somebody, help!
2658         You can write however you want to.\n
Name: body_text, dtype: object

The dataset has 41717 posts. Some initial observations:

- As per `AcceptedAnswerId` a significant number of those questions have no accepted answers
- A significant number have not been viewed either (or there was no data for those posts). 
- Very few have a title, but all of them have some body text. This doesn't seem right. Shouldn't all posts have titles?

Let's explore a bit more and start cleaning up the data too

## Data Cleaning

In [12]:
writers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41717 entries, 0 to 41716
Data columns (total 24 columns):
Unnamed: 0               41717 non-null int64
Id                       41717 non-null int64
PostTypeId               41717 non-null int64
AcceptedAnswerId         4971 non-null float64
CreationDate             41717 non-null object
Score                    41717 non-null int64
ViewCount                9674 non-null float64
Body                     41631 non-null object
OwnerUserId              38833 non-null float64
LastEditorUserId         13033 non-null float64
LastEditorDisplayName    985 non-null object
LastEditDate             13941 non-null object
LastActivityDate         41717 non-null object
Title                    9674 non-null object
Tags                     9674 non-null object
AnswerCount              9674 non-null float64
CommentCount             41717 non-null int64
FavoriteCount            3981 non-null float64
ClosedDate               1232 non-null object
Conten

In [13]:
writers["PostTypeId"].value_counts()

2    31308
1     9674
5      363
4      363
7        4
6        3
3        2
Name: PostTypeId, dtype: int64

As per the schema, `1` corresponds to a question, whereas `2` courresponds to answers. This would explain the discrepancy between the number of rows of `Title` and `body_text`.

Since we are only focusing on questions and answers, we will remove data corresponding to the rest of the Ids.

In [14]:
writers_clean = writers[writers["PostTypeId"].isin([1,2])]

writers_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40982 entries, 0 to 41716
Data columns (total 24 columns):
Unnamed: 0               40982 non-null int64
Id                       40982 non-null int64
PostTypeId               40982 non-null int64
AcceptedAnswerId         4971 non-null float64
CreationDate             40982 non-null object
Score                    40982 non-null int64
ViewCount                9674 non-null float64
Body                     40982 non-null object
OwnerUserId              38103 non-null float64
LastEditorUserId         12305 non-null float64
LastEditorDisplayName    977 non-null object
LastEditDate             13206 non-null object
LastActivityDate         40982 non-null object
Title                    9674 non-null object
Tags                     9674 non-null object
AnswerCount              9674 non-null float64
CommentCount             40982 non-null int64
FavoriteCount            3981 non-null float64
ClosedDate               1232 non-null object
Conten

We will start with a weak metric of assuming that a question's score relates to the quality of the question, and using this we will see which type of questions receive high scores.

First, we will match questions to their answers. We will roughly judge the quality of the text based on how well the accepted answers match the questions, for at least some of them.

In [ ]:
questions_with_accepted_answers = writers_clean[]